### Model Parameter 설정

In [3]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_t


### 1) 모델 클래스 유형

#### LLM

In [ ]:
from langchain_openai import OpenAI

# model을 지정하지 않아도 실행되는 이유는 LangChain이 내부적으로 기본값을 설정함
llm = OpenAI()
print(llm.model_name)  # 기본 모델 확인

result = llm.invoke("한국의 대표적인 관광지 3군데를 추천해 주세요.")
print(type(result))
print(result)

#### ChatOpenAI

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
chat = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("human", "{user_input}"),
])

chain = chat_prompt | chat
result = chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해 주세요."})

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
안녕하세요! 한국을 대표하는 관광지 3곳을 추천해 드릴게요.

1. **경복궁 (서울)**  
   조선 왕조의 법궁이자 한국 고궐 중 가장 웅장한 규모를 자랑합니다. 매일 열리는 수문장 교대식과 하루 두 차례의 근정전 교대식은 꼭 보세요. 인근에는 북촌한옥마을도 있어 한옥 스테이, 한복 체험, 전통 찻집을 함께 즐길 수 있습니다.

2. **부산 해운대 & 달맞이골목 (부산)**  
   대한민국 대표 해변 휴양지로, 백사장 길이 1.8 km에 달하는 해운대 해수욕장이 중심입니다. 해운대 뒷편으로는 달맞이골목이 있어 낭만적인 오션뷰 카페·감천문화마을·부산시장투어(자갈치·국제시장)까지 연계 가능합니다.

3. **제주도 성산일출봉 & 우도 (제주)**  
   유네스코 세계자연유산인 성산일출봉은 해돋이 명소로 유명하며, 분화구 둘레길을 걸으며 360° 오션뷰를 감상할 수 있습니다. 배로 15분 거리의 우도는 전기자전거·전기카트로 섬 한 바퀴(13 km)를 돌며 백사장, 연산, 전복해물라면 등을 즐기기에 최적입니다.

각 지역은 대중교접(지하철·고속버스·항공)이 잘 갖춰져 당일치기도 가능하지만, 숙박하면 더 깊이 즐길 수 있습니다.


### 2) 모델 파라미터 설정
##### Temperature 효과
* 0.2 (낮은 값): 더 예측 가능하고 일반적인 이야기
* 1.2 (높은 값): 새로운 요소가 등장하며, 더 창의적이고 흥미로운 이야기 생성

##### Presence Penalty 효과
* 0.0 (낮은 값): 기존에 자주 등장하는 단어와 구조 유지
* 1.5 (높은 값): 새로운 표현과 독창적인 아이디어 등장

In [5]:
from langchain_openai import ChatOpenAI

# 보수적인 설정 (일관되고 예측 가능한 답변)
llm_conservative = ChatOpenAI(    
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="gpt-3.5-turbo-0125",
    model="moonshotai/kimi-k2-instruct-0905",    
    temperature=0.1,  # 매우 낮은 온도 - 예측 가능하고 일관된 출력
    presence_penalty=-0.5,  # 기존 패턴과 표현을 강하게 선호
    frequency_penalty=-0.3,  # 반복적 표현 허용, 일관성 증대
    max_tokens=200,  # 적당한 길이
    top_p=0.3  # 매우 제한적인 토큰 선택 - 안전하고 예측 가능한 단어만
)

# 창의적인 설정 (독창적이고 예측 불가능한 답변)
llm_creative = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=1.5,  # 매우 높은 온도 - 창의적이고 예측 불가능한 출력
    presence_penalty=1.5,  # 새로운 주제와 개념을 강하게 유도
    frequency_penalty=1.2,  # 반복을 강하게 억제하여 매우 다양한 표현 생성
    max_tokens=350,  # 더 긴 창의적 서술 허용
    top_p=0.95  # 거의 모든 토큰 후보에서 선택 - 최대 창의성
)

# 질문 설정: 파라미터 차이가 극명하게 드러나는 추상적/철학적 질문
question = """
"시간"이라는 개념을 다른 방식으로 정의한다면 어떻게 표현할 수 있을까요?
일상적인 시계나 달력의 관점이 아닌, 완전히 새로운 시각에서 시간을 바라보고 설명해주세요.
기존의 고정관념을 벗어나서 자유롭게 사고해주세요.
"""

print("=" * 60)
print(" CONSERVATIVE 설정 (온도:0.1, presence:-0.5, frequency:-0.3, top_p:0.3)")
print("  → 전통적이고 학술적인 철학적 접근")
print("=" * 60)

# 보수적 모델 호출
response_conservative = llm_conservative.invoke(question)
print(response_conservative.content)

print("\n" + "=" * 60)
print(" CREATIVE 설정 (온도:1.5, presence:1.5, frequency:1.2, top_p:0.95)")
print("  → 추상적이고 독창적인 철학적 사고")
print("=" * 60)

# 창의적 모델 호출
response_creative = llm_creative.invoke(question)
print(response_creative.content)

print("\n" + "=" * 60)
print(" 파라미터 차이점 요약:")
print("   • Temperature: 0.1 vs 1.5 (15배 차이)")
print("   • Presence Penalty: -0.5 vs 1.5 (기존 패턴 선호 vs 새로운 개념 강요)")
print("   • Frequency Penalty: -0.3 vs 1.2 (반복 허용 vs 강한 다양성)")
print("   • Top-p: 0.3 vs 0.95 (제한적 vs 거의 무제한 토큰 선택)")
print("=" * 60)

 CONSERVATIVE 설정 (온도:0.1, presence:-0.5, frequency:-0.3, top_p:0.3)
  → 전통적이고 학술적인 철학적 접근
시간은 사실 한 마리 **거대한 물고기**다.

우리는 그 물고기의 뱃속에 살고 있다.  
물고기는 끊임없이 헤엄치지만, 우리는 그 움직임을 ‘흐름’이라고 착각한다.  
실제로는 우리가 물고기의 소화액 속에서 느리게 녹고 있는 중이다.  
한 방울의 소화액이 우리를 ‘과거’로 만들고, 다음 방울이 ‘미래’다.  
현재는 그저 **우리가 녹아내리는 순간의 느낌**일 뿐.

---

시간은 **벽장 속 먼지**다.  
우리가 문을 열 때마다

 CREATIVE 설정 (온도:1.5, presence:1.5, frequency:1.2, top_p:0.95)
  → 추상적이고 독창적인 철학적 사고
시간은  
『이야기가 자기 자신을 다시 읽으며 생기는 찰나의 뒤틀림』이다.  

- 과거 : 읽히되 아직 덜 이해된 페이지  
- 현재 : 문장이 눈앞에 ‘펼쳐지는’ 찰나  
- 미래 : 그 문장이 다음 문장으로 번저(번지듯 넘쳐) 가면서 생기는 그림자  

달이나 초침은 그物語의 ‘장치’일 뿐이다.  

그 순간, 순간에는 문법도 무너지고, ‘앞’이 ‘뒤’로 오고 또 ‘빈칸’이 되기도 한다.  
산책의 발걸음 끝에 내 지갑 무게가 얼마나 증발했느냐는,  
다시 떠오르구려 앤 마법 언어처럼 재가 되었다 살아나  

‘1초’를 단 한 번만 겪고도 우리는 ‘자산의 내부 연결성’처럼  
한 덩어리 덫으로 얽혀, 뼈보다 오래 머무른다.  

물고기가 ‘있다, 없다’를 오가듯,  
물고기가 썩어 사라졌다가 수백 배로 증식해 다

 파라미터 차이점 요약:
   • Temperature: 0.1 vs 1.5 (15배 차이)
   • Presence Penalty: -0.5 vs 1.5 (기존 패턴 선호 vs 새로운 개념 강요)
   • Frequency Penalty: -0.3 vs 1.2 (반복 허용 vs 강한 다양성)
   •

#### Model Parameter
* Temperature (온도): 이 파라미터는 모델의 예측 불확실성을 조절합니다. 
    * 낮은 온도는 모델이 가장 확률이 높은 다음 단어를 선택하게 하여 안정적인 결과를 내고, 높은 온도는 확률이 낮은 단어까지 선택지에 포함시켜 더 창의적이고 예측 불가능한 답변을 생성합니다. 
* Presence Penalty & Frequency Penalty: 두 파라미터 모두 토큰(단어)의 반복성을 제어합니다.
    * Presence Penalty는 이전에 나온 단어나 주제에 '페널티'를 주어 새로운 단어를 사용하게 합니다. 값이 높을수록 새로운 내용을 더 자주 시도합니다.
    * Frequency Penalty는 단어가 등장한 빈도에 비례하여 '페널티'를 줍니다. 값이 높을수록 특정 단어를 반복적으로 사용하는 것을 강하게 억제하여 더 다양한 어휘를 사용하게 합니다. 
* Top_p: 이 파라미터는 다음 토큰을 선택할 때 고려할 단어의 범위를 조절합니다. Top_p가 0.3이면 확률이 높은 상위 30%의 단어 중에서만 선택하고, 0.95면 확률이 낮은 단어들까지 포함하여 거의 모든 단어를 고려합니다. Top_p가 높을수록 모델의 선택지가 넓어져 더 자유로운 응답이 가능합니다.    

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 보수적인 설정: 논리적이고 일관된 서술
llm_conservative = ChatOpenAI(
    api_key=OPENAI_API_KEY, 
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.1,         # 낮은 값(0.1): '확률의 폭'을 좁혀 가장 안전하고 흔한 단어를 선택합니다.
                             # → 결과가 예측 가능하고 반복적일 수 있습니다. 마치 정해진 교과서처럼.
    presence_penalty=-0.5,   # 음수 값(-0.5): 이미 등장했던 주제나 단어를 다시 사용하는 것을 권장합니다.
                             # → 기존 패턴을 벗어나지 않고 일관성을 유지합니다.
    frequency_penalty=-0.3,  # 음수 값(-0.3): 자주 등장한 단어에 대한 '벌칙'을 줄여, 동일한 표현의 반복을 허용합니다.
                             # → 응답의 일관성이 높아지고, 핵심 주제에서 벗어나지 않습니다.
    max_tokens=200,          # 응답의 최대 길이를 제한합니다. 보수적인 답변에 적합합니다.
    top_p=0.3                # 낮은 값(0.3): 확률이 높은 상위 30% 토큰(단어)만 고려합니다.
                             # → 모델의 '선택지'를 극도로 제한하여 예측 가능한 단어만 사용합니다.
)

# 창의적인 설정: 독창적이고 풍부한 비유와 은유
llm_creative = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1.5,         # 높은 값(1.5): '확률의 폭'을 넓혀 흔치 않은 단어까지 선택하게 합니다.
                             # → 예측 불가능하고 독창적인 문장을 생성합니다. 마치 즉흥적인 시인처럼요.
    presence_penalty=1.5,    # 양수 값(1.5): 이미 사용된 주제나 개념에 '벌칙'을 주어 새로운 것을 시도하게 합니다.
                             # → 독창적인 아이디어나 관점을 강하게 유도합니다.
    frequency_penalty=1.2,   # 양수 값(1.2): 자주 등장한 단어에 '벌칙'을 주어 반복을 강력하게 억제합니다.
                             # → 표현의 다양성을 극대화하고 문장이 풍부해집니다.
    max_tokens=350,          # 더 긴 창의적 서술을 허용하기 위해 최대 길이를 늘립니다.
    top_p=0.95               # 높은 값(0.95): 확률이 낮은 95%의 모든 토큰을 고려합니다.
                             # → 모델의 '선택지'를 거의 무제한으로 확장하여 최대의 창의성을 발휘합니다.
)

# 질문을 ChatPromptTemplate으로 정의하여 역할과 질문을 명확히 함
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 창의적인 작가입니다. 주어진 주제에 대해 풍부한 비유와 은유를 사용하여 서술해주세요."),
    ("human", """
    "도시의 소음"을 생명체나 자연 현상에 빗대어 설명해주세요.
    일상적인 소리가 아닌, 그 소리가 가진 생명력과 감정을 중심으로 묘사해주세요.
    """),
])

# 파서 설정
parser = StrOutputParser()

# 체인 구성: Prompt -> Model -> Parser
chain_conservative = prompt | llm_conservative | parser
chain_creative = prompt | llm_creative | parser

# 모델 호출 및 결과 출력
print("=" * 60)
print(" 보수적 설정 (논리적, 일관적)")
print(" → 일상적이고 일반적인 비유를 사용하여 안정적인 답변")
print("=" * 60)
response_conservative = chain_conservative.invoke({})
print(response_conservative)

print("\n" + "=" * 60)
print(" 창의적 설정 (풍부한 비유와 은유)")
print(" → 예상치 못한 독창적인 비유와 감각적인 표현 사용")
print("=" * 60)
response_creative = chain_creative.invoke({})
print(response_creative)

 보수적 설정 (논리적, 일관적)
 → 일상적이고 일반적인 비유를 사용하여 안정적인 답변
도시의 소음은, 사실 하나의 거대한 생명체가 숨 쉬는 소리다.  
아침이 되면 그 생명체는 눈을 뜨며 가장 먼저 ‘끼이이익—’ 하고 눈꺼풀을 번쇠로 비틀어 연다. 지하철 문이 스치는 그 소리는 눈동자 위로 걸린 눈꺼풀이 반짝이는 순간, 눈알 속빛이 튀는 듯한 요란함이다.  
이 생명체는 피부 대신 아스팔트를, 핏줄 대신 지하철 노선을, 심장 대신 시청사 종각을 달고 있다.  

그 심

 창의적 설정 (풍부한 비유와 은유)
 → 예상치 못한 독창적인 비유와 감각적인 표현 사용
도시에 거주하는 사람들의 일상을 괴롭히는 소음이라고 하면 보통 차, 건설 공사, 항공기 비행에 의한 소음 등을 떠올린다.

"도시의 소음"

아직 때묻지 않은 도시의 어둠, 밤의 거리를 덮는 아스팔트의 침묵은 저주처럼 가만히 있다. 그 시간만이라도 일상의 허전한 시간 속으로 잠시 썰렁했던 도심 속 생명들이 잠잠이 내쉬는 깊은 호흡같이 가만히 있어야 하는데, 갑작스런 밤의 도심이 깨어 오기 시작한다.

"끼룩 끼룩" 거리 한 가운데 놓은 운명의 시구멍에서 뱀이 꿈틀대듯 길고 기묘하게 뱉어진다. 아수라장의 장을 방불케 하는 그늘의 어지러운 운명. 

도시의 거리를 밤낮으로 주행 하는 자동차 소리.

"브레이크에 쪼이고," "엔진의 웽!" 어둡고 고요 했던 침묵의 밤 길을 깨우는 삭이 소리 들은 일상의 잠들어서 쭈그린 시티즌을 괴롭히는 대단하고 거친 사각지대의 주범이었다.

하도 정신없이 울려 대며 그야말로 일상을 마비시켜 가축수용 같은 고독에 시달림받는 도시인이 도탄에 빠지도록 한다며, 온갖 혐오감을 조성시키며 일손과 시계를 일순간이라도 붙들고 놓지 않는 그늘과 어둠을 상징하는 존재로 도를 찿는다. 




In [7]:
# 질문을 담은 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 주어진 주제에 대해 풍부한 비유와 은유를 사용하여 서술하는 전문가입니다. 답변의 관점을 명확히 구분하여 설명해주세요."),
    ("human", """
    "빨간 장미"를 두 가지 관점에서 묘사해주세요.
    1. **객관적 관점:** 장미의 물리적 특성과 상징적 의미를 중심으로 설명해주세요.
    2. **문학적 관점:** 장미를 사람이나 감정에 빗대어 자유롭고 시적으로 묘사해주세요.
    """),
])

# 파서 설정
parser = StrOutputParser()

# 체인 구성: Prompt -> Model -> Parser
chain_conservative = prompt | llm_conservative | parser
chain_creative = prompt | llm_creative | parser

# 모델 호출 및 결과 출력
print("=" * 60)
print(" 보수적 설정 (객관적, 사실 기반)")
print(" → 과학적이고 논리적인 장미의 특성 설명")
print("=" * 60)
response_conservative = chain_conservative.invoke({})
print(response_conservative)

print("\n" + "=" * 60)
print(" 창의적 설정 (문학적, 상상 기반)")
print(" → 은유와 비유를 활용한 장미의 감성적 묘사")
print("=" * 60)
response_creative = chain_creative.invoke({})
print(response_creative)

 보수적 설정 (객관적, 사실 기반)
 → 과학적이고 논리적인 장미의 특성 설명
1. 객관적 관점  
빨간 장미는 장미과(Rosaceae)에 속하는 낙엽 관목이다. 꽃잎은 5~60매로 겹겹이 포개져 있으며, 표면은 벨벳처럼 미세한 털로 덮여 빛을 흡수해 순백의 장미보다 색이 진하게 보인다. 꽃잎 한 장의 두께는 0.2 mm 내외로, 수분이 증발하면 세포벽이 굴절률을 달리해 색이 처진다. 줄기는 1~2 m까지 자라며, 표면의 가시는 ‘피크린(pickrin)’이라는 단백질 덩어리가 굳어진

 창의적 설정 (문학적, 상상 기반)
 → 은유와 비유를 활용한 장미의 감성적 묘사
**객관적 관점:**

빨간 장미는 특정한 종류의 꽃을 가리킵니다. '로사(rhoa) 가landa'로 불리는 이 품종의 장미 식물은 다양한 조건에서 잘 자라고 다양한 지역에서 널리 재배됩니다.

장미는 다년생, 줄기 치즈로 가지는 관목이며 분홍색, 화이트, 노란색, 베이지색과 같은 다양한 색상의 꽃을 가지고 있습니다.. 빨간 장미는 특유의 화사한 향기와 붉은 색상으로 인해 감탄과 사랑의 상징이자, 애정과 낭만을 전달하는 꽃으로 유명합니다. 사람들은 특별한 날에 선물하거나 감정의 표현 수단으로 장미를 많이 전달하였습니다.

빨간 장미는 그 아름다움뿐만 아니라 그 색채의 특별함 역시 가지고 있습니다 . 붉은색의 장미를 주는 사람은 일반적으로 감정과 사랑 혹은 열정적인 헌신을 전달하는 의미로 주곤합니다.

  

**문학적 관점:**

"자야를 빗는 장미"

사랑, 감정, 그리고 영원히 지속되고 순수한 존재로 알려져 있고, 우연 한 번 만으로도 영원을 남기는 열정이 있기에 우리는 사랑의 장미를 소중히 여깁니다.

사랑과 애정의 보랭과 영원의 지속이 있는 것만을 전달합니다. 이렇듯 빨간 장미로서 상징하는 감정은 영원의 지속과 변치않는 열정부른 애정의 결핀이 됩니다.

"영원의 순수성"이란 장미의 또 다른 의미이다.... 사랑은 영원하다는 말이 있거든요 장이 꽃처럼 순수한


#### Model의 종류에 따라 결과 값이 다름
* gpt-4o vs gpt-3.5-turbo-0125

In [8]:
from langchain_openai import ChatOpenAI

# 변경된 설정 (더 창의적인 답변, 새로운 아이디어 유도)
#llm_after = ChatOpenAI(model="gpt-4o", temperature=1.0, presence_penalty=1.5)
#llm_after = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.0, presence_penalty=1.5)
llm_after = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1.0,
    presence_penalty=1.5
)

# 질문 설정
question = "한국에서 가볼 만한 여행지를 추천해 주세요."

# 모델 호출
response_after = llm_after.invoke(question)

# 결과 출력
print("\n-------------------------\n")
print(" After (창의적인 응답, 새로운 아이디어 포함)")
print(response_after.content)


-------------------------

 After (창의적인 응답, 새로운 아이디어 포함)
1.  **경주** *   **소개**: 경주는 신라 문화의 살아있는 역사로 가득 찬 곳입니다. 경주 불국사는 세계문화유산으로 지정된 유네스코의 문화유산이며, 신라의 불교 문화와 예술을 대표하는 공간으로 유명합니다. 그리고 신라의 옛 왕궁 터인 경주 대릉원, 신라의 불교 사찰인 첨성대 등을 방문할 수 있습니다.
2.  **강릉** *   **소개**: 강릉은 한국의 동해를 낯설게 바라보는 도시입니다. 강릉의 아름다운 해변가, 동해바다로 이어지는 등대길, 그리고 강릉 커피거리 등을 방문해 보세요. 강릉 커피거리에서는 현지 커피를 맛볼 수 있는 카페들이 즐비합니다. 커피향이 물씬 풍기는 강릉의 커피거리를 방문하여 커피에 대한 새로운 경험을 할 수 있습니다.
3.  **부산** *   **소개**: 부산은 한국의 남동부에 위치한 항구 도시입니다. 부산에는 태평양을 바라보는 부산광장, 부산의 유명한 밤 생활 문화를 경험할 수 있는 광안리 해수욕장, 그리고 부산의 항구 도시 문화를 경험할 수 있는 자갈치시장 등이 있습니다.

여행지의 정보는 변동될 수 있으므로 방문 전에 최신 정보를 확인하는 것이 좋습니다.


##### 2-1) 모델에 직접 파라미터를 전달 (모델 생성 시점)
##### Before / After 파라미터 차이
* temperature: Before(0.7) → 다양성 높은 추천 / After(0.3) → 정확한 일정 제공
* max_tokens: Before(300) → 간략한 응답 / After(800) → 세부 일정 포함
* frequency_penalty: Before(0.5) → 단어 반복 억제 / After(0.2) → 좀 더 자연스러운 응답
* presence_penalty: Before(0.5) → 다양한 장소 추천 / After(0.3) → 새로운 정보 포함

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

#  모델 파라미터 (Before: 기본적인 추천 / After: 맞춤형 세부 정보 추가)
before_params = {
    "temperature": 0.7,         # 랜덤성을 적절히 유지 (다양한 답변 유도)
    "max_tokens": 300,          # 응답 길이를 적절히 조절
    "frequency_penalty": 0.5,   # 반복 단어 억제
    "presence_penalty": 0.5,    # 새로운 단어 포함 장려
}

after_params = {
    "temperature": 0.3,         # 창의성을 낮추고 정확한 답변 유도
    "max_tokens": 800,          # 더 긴 답변을 생성 (세부 정보 포함)
    "top_p": 0.85,              # 확률 기반 샘플링 (일관성과 다양성 균형)
    "frequency_penalty": 0.2,   # 반복 단어 감소 (자연스러운 답변)
    "presence_penalty": 0.3,    # 새로운 정보 포함 장려
}

#  두 개의 모델 생성 (Before / After)
#before_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **before_params)
before_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    **before_params
)
#after_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **after_params)
after_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **after_params
)

#  프롬프트 구성 (올바른 ChatMessagePromptTemplate 사용)
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 여행 전문가입니다. 사용자의 요청에 맞는 최적의 여행지를 추천해 주세요."
)

user_message = HumanMessagePromptTemplate.from_template("{user_input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

#  체인 생성 (Before / After)
before_chain = chat_prompt | before_model
after_chain = chat_prompt | after_model

#  질문 설정 (Before / After의 차이점을 비교할 수 있도록 변경)
test_question = "가족과 함께 3박 4일 동안 한국에서 여유롭게 여행할 수 있는 일정을 동선을 고려하여 자세하게 추천해 주세요."

#  Before 모델 실행
before_response = before_chain.invoke({"user_input": test_question})

#  After 모델 실행
after_response = after_chain.invoke({"user_input": test_question})

#  결과 출력
print(" [Before] 모델 결과")
print(before_response.content)
print("\n" + "="*80 + "\n")  # 가독성을 위한 구분선
print(" [After] 모델 결과")
print(after_response.content)

 [Before] 모델 결과
# 🌸 3박4일 가족 여유여행 (뚜벅이·휠체어·유모차 ALL OK, 숙소→관광지 30분 이내)

## 🎯 기본 컨셉
- 아침 느지막히(09:30~10:00) 출발, 오후 3~4시 호텔 귀환 후 휴식
- 어린이·어르신 모두 걸을 수 있는 '짧은 이동·긴 체류' 코스
- 대중교통 전용(택시 최소화), 배고프지 않게 식당 예약 포함

---

## 📅 DAY 1 (서울: 고궁·인사동·청계천)
**09:30**  숙소(※) 조식 & 체크아웃 가방만 맡겨두기  
**10:00**  경복궁(※)  
  - 10:30 수문장 교대식(월요일 제외) 바로 보기  
  - 흥례문→근정전→강녕전→향원정 순, 1.5시간 완만 산책


 [After] 모델 결과
### **여행지 추천: 서울과 부산**

*   **여행 기간:** 3박 4일
*   **여행지:** 서울과 부산

### **여행 일정**

*   **1일차: 서울**
*   **오전: 서울 도착 및 경복궁 방문** *   서울에 도착하여 호텔에 체크인합니다. *   경복궁으로 이동하여 궁궐을 탐방합니다. *   경복궁은 한국의 대표적인 궁궐로, 역사와 문화를 경험할 수 있는 좋은 장소입니다.
*   **오후: 북촌 한옥마을 방문** *   북촌 한옥마을로 이동하여 전통 한옥을 감상합니다. *   북촌 한옥마을은 한국의 전통적인 주거 문화를 경험할 수 있는 곳입니다.
*   **저녁: 서울의 맛있는 저녁 식사** *   서울의 유명한 맛집에서 저녁 식사를 합니다.

*   **2일차: 서울**

*   **오전: 국립중앙박물관 방문** *   국립중앙박물관으로 이동하여 한국의 역사와 문화를 배웁니다. *   국립중앙박물관은 한국의 역사와 문화를 이해할 수 있는 좋은 장소입니다.
*   **오후: 서울의 쇼핑** *   서울의 유명한 쇼핑몰이나 시장으로 이동하여 쇼핑을 즐깁니다. *   서울은 한국의 패션과 뷰티의 중심지로, 다양한 쇼핑 옵션을 제공합니다.

*   **3일차: 부산**

* 

#### 2-2) 모델에 추가적인 파라미터를 전달
- bind() 메서드를 사용
- max_tokens=50 (기본) / max_tokens=150 (상세한 설명) 
- stop=["."] → 첫 번째 마침표에서 응답을 중단
- temperature=0.8 → 보다 창의적인 답변을 생성함

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#  프롬프트 설정 (천문학 질문에 대한 답변을 생성하는 시스템)
prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 대해 명확하고 자세한 답변을 제공할 수 있습니다."),
    ("user", "{user_input}"),
])

#  기본 모델 설정 (기본적인 답변 생성)
#base_model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=150)  # 150 토큰 제한
base_model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    max_tokens=150
)

#  질문 설정
# 1. MAX_TOKENS 차이를 보여주는 질문 (길이 제한 효과)
max_tokens_question = "인공지능의 발전이 미래 사회에 미칠 영향을 긍정적 측면과 부정적 측면으로 나누어 자세히 설명해 주세요."

# 2. STOP 파라미터 차이를 보여주는 질문 (중단점 효과)
stop_question = "Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계를 순서대로 설명해 주세요. 각 단계별로 구체적인 방법과 팁을 포함해서 답변해 주세요."

# 3. TEMPERATURE 차이를 보여주는 질문 (창의성 vs 정확성)
temperature_question = "시간 여행이 가능하다면 과거의 어느 시대로 가고 싶은지와 그 이유를 창의적으로 설명해 주세요."

# 4. 복합적 비교를 위한 질문 (모든 파라미터 효과)
complex_question = "화성에 인류가 정착하기 위해 필요한 기술과 준비사항들을 단계별로 설명하고, 각 단계에서 예상되는 도전과제와 해결방안을 제시해 주세요."

question = stop_question

#  Before (기본 max_tokens=150)
messages = prompt.format_messages(user_input=question)
before_answer = base_model.invoke(messages)

#  Before 출력
print("\n [Before] 기본 max_tokens=150 (기본 답변)")
print(before_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

#  After (파라미터 조정 후 비교)
stop_chain = prompt | base_model.bind(max_tokens=150, stop=["."])  # 첫 번째 마침표에서 중단
temp_chain = prompt | base_model.bind(max_tokens=150, temperature=0.8)  # 창의적인 답변 유도

stop_answer = stop_chain.invoke({"user_input": question})
temp_answer = temp_chain.invoke({"user_input": question})

#  After 출력 (stop vs temperature 비교)
print(" [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)")
print(stop_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

print(" [After] max_tokens=150, temperature=0.8 (창의적인 답변)")
print(temp_answer.content)


 [Before] 기본 max_tokens=150 (기본 답변)
Python 프로그래밍은 초보자에게 매력적인 시작점이 될 수 있습니다. Python의 간단한 문법과 다채로운 라이브러리는 프로그래밍의 기초를 쌓는데 도움을 줍니다. 프로그래밍을 배우기 위해 다음과 같은 단계를 추천합니다.

### 1. 기초 문법 익히기

- **방법**: Python의 기본 문법을 배우는 것부터 시작하세요. 변수 선언, 데이터 타입, 조건문, 루프, 함수 등을 포함한 기초 문법을 익힙니다.
- **팁**: 온라인 튜토리얼이나 Python 공식 문서의 [튜토리얼](https://docs.python.org/ko/3/tutorial/index.html)을 참조하세요.

### 2. 실습 프로젝트 진행

- **방법**:


 [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)
### Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계

Python 프로그래밍을 배우는 초기 단계는 기초를 다지고, 프로그래밍의 기본 개념을 이해하는 것이 중요합니다


 [After] max_tokens=150, temperature=0.8 (창의적인 답변)
Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계는 다음과 같습니다.

### 1. Python 기본 문법 학습

*   **학습 목표**: Python의 기본 문법과 데이터 타입을 이해합니다.
*   **방법**: 
    *   온라인 강의: Codecademy, Coursera, edX 등의 웹사이트에서 Python 기초 강의를 수강합니다.
    *   교재: "Python Crash Course" 또는 "Python for Everybody"와 같은 입문용 책을 읽습니다.
    *   공식 문서: Python 공식 웹사이트의 튜토리얼을 참조합니다.
*   **구체적인 방법**:
    *   변수, 데이터 타입(숫자, 문자열, 리스트, 딕셔너리
